In [ ]:
!pip install transformers==3.3.1
!pip install timm

     |████████████████████████████████| 1.1MB 23.3MB/s 
     |████████████████████████████████| 1.2MB 53.0MB/s 
     |████████████████████████████████| 901kB 51.1MB/s 
     |████████████████████████████████| 3.0MB 58.2MB/s 
     |████████████████████████████████| 348kB 25.0MB/s 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("ggplot")

import timm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models

from zipfile import ZipFile
import os
import time
from shutil import copy2
from torch.utils.data import DataLoader
from transformers import AdamW,get_linear_schedule_with_warmup,AutoModel,AutoTokenizer
from PIL import Image
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report

In [ ]:
extract_path = '/content/drive/MyDrive/Colab Notebooks/Datasets and weights/Tamil_troll_memes/training_img.zip'
with ZipFile(extract_path, 'r') as zipObj:
   zipObj.extractall()

In [ ]:
def move_data(start,troll,not_troll):
  for img_name in os.listdir(start):
    src = os.path.join(start,img_name)
    if img_name.startswith('N'):
      copy2(src,not_troll)
    else:
      copy2(src,troll)

os.mkdir('Troll')
os.mkdir('Non_troll')
src = '/content/uploaded_tamil_memes'
move_data(src,'/content/Troll','/content/Non_troll')

def split_data(start,train,val,split):
  for i, img_name in enumerate(os.listdir(start)):
    src = os.path.join(start,img_name)
    if i < split:
      copy2(src,val)
    else:
      copy2(src,train)

os.mkdir('Train')
os.mkdir('Val')
split_data('/content/Troll','/content/Train','/content/Val',128)
split_data('/content/Non_troll','/content/Train','/content/Val',101)

In [ ]:
class TamilDataset(torch.utils.data.Dataset):
  def __init__(self,df,tokenizer,max_len,path,transforms=None):
    self.data_dir = path
    self.df = df
    self.tokenizer = tokenizer
    self.transforms = transforms
    self.max_len = max_len

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,index):
    img_name, captions = self.df.iloc[index]
    img_path = os.path.join(self.data_dir,img_name)
    labels = 0 if img_name.startswith('N') else 1
    img = Image.open(img_path).convert('RGB')

    if self.transforms is not None:
      img = self.transforms(img)

    encoding = self.tokenizer.encode_plus(
        captions,
        add_special_tokens=True,
        max_length = self.max_len,
        return_token_type_ids = False,
        padding = 'max_length',
        return_attention_mask= True,
        return_tensors='pt',
        truncation=True
    )

    return {
        'image' : img,
        'text' : captions,
        'input_ids' : encoding['input_ids'].flatten(),
        'attention_mask' : encoding['attention_mask'].flatten(),
        'label' : torch.tensor(labels,dtype=torch.float)
    } 

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets and weights/Tamil_troll_memes/train_captions.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)

train_df_data = []
val_df_data = []
for img_name in os.listdir('/content/Train'):
  ind = list(df[df['imagename'] == img_name].index)[0]
  train_df_data.append([img_name,df['captions'].iloc[ind]])

for img_name in os.listdir('/content/Val'):
  ind = list(df[df['imagename'] == img_name].index)[0]
  val_df_data.append([img_name,df['captions'].iloc[ind]])

train_df = pd.DataFrame(train_df_data,columns=['img_name','captions'])
val_df = pd.DataFrame(val_df_data,columns=['img_name','captions'])

In [ ]:
def create_data_loader(df,tokenizer,max_len,batch_size,mytransforms,path,shuffle):
  ds = TamilDataset(
      df,
      tokenizer,
      max_len,
      path,
      mytransforms
  )

  return DataLoader(ds,
                    batch_size = batch_size,
                    shuffle=False,
                    num_workers=2)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
PRE_TRAINED_MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
my_trans = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),     
])

BATCH_SIZE = 16
MAX_LEN = 128
train_data_loader = create_data_loader(train_df,tokenizer,MAX_LEN,BATCH_SIZE,my_trans,'/content/Train',True)
val_data_loader = create_data_loader(val_df,tokenizer,MAX_LEN,BATCH_SIZE,my_trans,'/content/Val',False)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__()
        self.bert = AutoModel.from_pretrained('bert-base-multilingual-cased')
        self.fc1 = nn.Linear(768,128)
        self.fc2 = nn.Linear(128,1)
        self.gelu = nn.ReLU()
        self.drop = nn.Dropout(0.2)
    
    def forward(self,inp,atten):
        _, x = self.bert(inp,atten)
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.gelu(x)
        
        return x

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
        self.incep4 = timm.create_model('inception_v4',pretrained=True)
        self.incep4.last_linear = nn.Linear(1536,768)
        self.fc1 = nn.Linear(768,127)
        self.leak = nn.LeakyReLU()
        self.fc2 = nn.Linear(128,1)
        self.drop = nn.Dropout(0.2)

    def forward(self,img,t_out):
        img = self.incep4(img)
        img = self.fc1(img)
        img = self.leak(img)
        img = self.drop(img)
        out = torch.cat((img,t_out),dim=1)
        out = self.fc2(out)

        return out

In [ ]:
def epoch_time(start_time,end_time):
	elapsed_time = end_time - start_time
	elapsed_mins = int(elapsed_time/60)
	elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
	return elapsed_mins,elapsed_secs

In [ ]:
def train_epoch(gen,disc,data_loader,gen_loss,disc_loss,gen_opt,disc_opt,device,n_examples):
    gen_losses = []
    disc_losses = []
    correct_preds = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        img = data['image'].to(device)
        label = data['label'].to(device)

        #train disc
        disc_opt.zero_grad()
        with torch.no_grad():
            fake = gen(input_ids,attention_mask)
        disc_fake_labels = disc(img,fake.detach())
        disc_fake_loss = disc_loss(disc_fake_labels, torch.zeros_like(disc_fake_labels))
        
        real_labels = torch.unsqueeze(label,dim=1)
        disc_real_labels = disc(img,real_labels)
        disc_real_loss = disc_loss(disc_real_labels, torch.ones_like(disc_real_labels))
        total_disc_loss = (disc_fake_loss + disc_real_loss) / 2
        total_disc_loss.backward(retain_graph=True)
        disc_opt.step()
        disc_losses.append(total_disc_loss.item())

        #train gen
        gen_opt.zero_grad()
        gen_pred = gen(input_ids,attention_mask)
        disc_pred = disc(img,gen_pred)
        total_gen_loss = gen_loss(disc_pred,torch.ones_like(disc_pred))
        total_gen_loss.backward()
        gen_opt.step()
        gen_losses.append(total_gen_loss.item())

        sig_outs = torch.sigmoid(torch.squeeze(gen_pred,dim=1))
        preds = [0 if x < 0.5 else 1 for x in sig_outs]
        preds = torch.tensor(preds).to(device)
        correct_preds += torch.sum(preds == label)

    return correct_preds / n_examples, np.mean(gen_losses), np.mean(disc_losses)

In [ ]:
def val_epoch(gen,data_loader,device,n_examples):
    correct_preds = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        img = data['image'].to(device)
        label = data['label'].to(device)

        with torch.no_grad():
            gen_preds = gen(input_ids,attention_mask)
        
        preds = [0 if x < 0.5 else 1 for x in torch.sigmoid(gen_preds)]
        preds = torch.tensor(preds).to(device)
        correct_preds += torch.sum(preds == label)

    return correct_preds / n_examples

In [ ]:
EPOCHS = 5

gen = Generator().to(device)
disc = Discriminator().to(device)

gen_opt = AdamW(gen.parameters(), lr=2e-5, correct_bias=False)
disc_opt = optim.Adam(disc.parameters(), lr = 0.0002)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/inceptionv4-8e4777a0.pth" to /root/.cache/torch/hub/checkpoints/inceptionv4-8e4777a0.pth


In [ ]:
gen_loss = nn.BCEWithLogitsLoss().to(device)
disc_loss = nn.BCEWithLogitsLoss().to(device)

In [ ]:
from collections import defaultdict

history = defaultdict(list)
best_accuracy = 0
for epoch in range(EPOCHS):
 
 
  start_time = time.time()
  train_acc,gen_losse,disc_losse = train_epoch(
      gen,
      disc,
      train_data_loader,
      gen_loss,
      disc_loss,
      gen_opt,
      disc_opt,
      device,
      2071
  )
   
  
  val_acc = val_epoch(
      gen,
      val_data_loader,
      device,
      229
  )
  
  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'Gen Loss {gen_losse} Disc Loss {disc_losse}')
  print(f'Train accuracy {train_acc}')
  print(f'Val accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['gen_loss'].append(gen_losse)
  history['disc_loss'].append(disc_losse)
  history['val_acc'].append(val_acc)

Epoch: 01 | Epoch Time: 3m 40s
Gen Loss 0.6930558814452245 Disc Loss 0.6958770504364601
Train accuracy 0.5572187304496765
Val accuracy 0.5589519739151001

Epoch: 02 | Epoch Time: 3m 40s
Gen Loss 0.6949315291184646 Disc Loss 0.691616611297314
Train accuracy 0.5572187304496765
Val accuracy 0.5589519739151001

Epoch: 03 | Epoch Time: 3m 41s
Gen Loss 0.6971177188249734 Disc Loss 0.6879663091439467
Train accuracy 0.5572187304496765
Val accuracy 0.5589519739151001

Epoch: 04 | Epoch Time: 3m 40s
Gen Loss 0.7004121060554798 Disc Loss 0.6843227771612314
Train accuracy 0.5572187304496765
Val accuracy 0.5589519739151001

Epoch: 05 | Epoch Time: 3m 40s
Gen Loss 0.7054866460653452 Disc Loss 0.6808523654937744
Train accuracy 0.5572187304496765
Val accuracy 0.5589519739151001



In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets and weights/Tamil_troll_memes/test_captions.csv')
df_test.drop('Unnamed: 0',axis=1,inplace=True)
extract_path = '/content/drive/MyDrive/Colab Notebooks/Datasets and weights/Tamil_troll_memes/test_img.zip'
with ZipFile(extract_path, 'r') as zipObj:
   zipObj.extractall()

In [ ]:
test_data_loader = create_data_loader(df_test,tokenizer,MAX_LEN,BATCH_SIZE,my_trans,'/content/test_img',False)

In [ ]:
def get_predictions(model,data_loader, device):
    model = model.eval()
    f_preds = []
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            image = d['image'].to(device)
            outputs = model(
                input_ids,
                attention_mask,
            )
            sig_outs = torch.sigmoid(outputs)
            preds = [0 if x < 0.5 else 1 for x in sig_outs]
            for j in preds:
                f_preds.append(j)
    
    return f_preds

In [ ]:
submission_preds = get_predictions(gen,test_data_loader,device)

In [ ]:
df_org = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets and weights/Tamil_troll_memes/gold_labels_for_test.csv')
def f(x):
    if x == 'troll':
        return 1
    return 0

df_org['label'] = df_org['label'].apply(lambda x : f(x))

In [ ]:
print(classification_report(df_org['label'],submission_preds,target_names=['Non-Troll','Troll']))